In [7]:
#import pyfair
from pyfair.model.model import FairModel
from pyfair.model.meta_model import FairMetaModel

import matplotlib
%matplotlib inline

import numpy as np
import pandas as pd

import pyfair

import json

import uuid

In [8]:
# Create using LEF (PERT), PL, (PERT), and SL (constant)
model1 = pyfair.FairModel(name="Regular Model 1", n_simulations=10_000)
model1.input_data('Loss Event Frequency', low=20, mode=100, high=900)
model1.input_data('Primary Loss', low=3_000_000, mode=3_500_000, high=5_000_000)
model1.input_multi_data('Secondary Loss', {
    'Reputational': {
        'Secondary Loss Event Frequency': {'constant': 4000}, 
        'Secondary Loss Event Magnitude': {'low': 10, 'mode': 20, 'high': 100},
    },
    'Legal': {
        'Secondary Loss Event Frequency': {'constant': 2000}, 
        'Secondary Loss Event Magnitude': {'low': 10, 'mode': 20, 'high': 100},        
    }
})
model1.calculate_all()
meta1 = FairMetaModel(name='My Meta Model!', models=[model1, model1])
meta1.calculate_all()
print(model1.to_json())


{
    "Loss Event Frequency": {
        "low": 20,
        "mode": 100,
        "high": 900,
        "gamma": 4
    },
    "Primary Loss": {
        "low": 3000000,
        "mode": 3500000,
        "high": 5000000,
        "gamma": 4
    },
    "multi_Secondary Loss": {
        "Reputational": {
            "Secondary Loss Event Frequency": {
                "constant": 4000
            },
            "Secondary Loss Event Magnitude": {
                "low": 10,
                "mode": 20,
                "high": 100
            }
        },
        "Legal": {
            "Secondary Loss Event Frequency": {
                "constant": 2000
            },
            "Secondary Loss Event Magnitude": {
                "low": 10,
                "mode": 20,
                "high": 100
            }
        }
    },
    "name": "Regular Model 1",
    "n_simulations": 10000,
    "random_seed": 42,
    "model_uuid": "880edb68-a675-11e9-a220-f26e0bbd6dbc",
    "type": "FairModel",
    "crea

In [6]:
import json
import pathlib
import sqlite3

import pandas as pd

#from . import FairException
from pyfair.utility.fair_exception import FairException

from pyfair.model.meta_model import FairMetaModel
from pyfair.model.model import FairModel


class FairDatabase(object):
    '''JSON database antipattern.'''
    
    def __init__(self, path):
        self._path = pathlib.Path(path)
        self._initialize()
    
    def _initialize(self):
        '''Initialize if necessary.'''
        with sqlite3.connect(self._path) as conn:
            conn.execute('''CREATE TABLE IF NOT EXISTS model (uuid string, 
                                                              name string, 
                                                              creation_date text NOT NULL,
                                                              json string NOT NULL,
                                                              CONSTRAINT model_pk PRIMARY KEY (uuid));''')
            conn.execute('''CREATE TABLE IF NOT EXISTS results (uuid string,
                                                                mean real NOT NULL, 
                                                                stdev real NOT NULL, 
                                                                min real NOT NULL, 
                                                                max real NOT NULL, 
                                                                CONSTRAINT results_pk PRIMARY KEY (uuid));''')

    
    def _dict_factory(self, cursor, row):
        '''Convenience function for sqlite'''
        # https://stackoverflow.com/questions/3300464/how-can-i-get-dict-from-sqlite-query
        d = {}
        for idx, col in enumerate(cursor.description):
            d[col[0]] = row[idx]
        return d
    
    def load(self, name_or_uuid):
        '''Take a name or UUID and dispatch to appropriate function'''
        # If it is a valid UUID
        try:
            uuid.UUID(name_or_uuid)
            model_or_metamodel = self._load_uuid(name_or_uuid)
        # If not a valid UUID, load by name
        except ValueError:
            model_or_metamodel = self._load_name(name_or_uuid)
        model_or_metamodel.calculate_all()
        return model_or_metamodel
    
    def _load_name(self, name):
        '''Load model or metamodel based on first item with that name.'''
        with sqlite3.connect(self._path) as conn:
            # Create SQlite fow factory
            conn.row_factory = self._dict_factory
            cursor = conn.cursor()
            # Search for models
            cursor.execute("SELECT uuid FROM model WHERE name = ?", (name,))
            result = cursor.fetchone()
            if not result:
                raise FairException('Name for model not found.')
            # Use model UUID query to load via _load_uuid function
            model = self._load_uuid(result['uuid'])
            return model
    
    def _load_uuid(self, uuid):
        '''Load model or metamodel based on ID'''
        # Get models and metamodels
        with sqlite3.connect(self._path) as conn:
            conn.row_factory = self._dict_factory
            cursor = conn.cursor()
            # Search for models
            cursor.execute("SELECT * FROM model WHERE uuid = ?", (uuid,))
            model_data = cursor.fetchone()
            if not model_data:
                raise FairException('UUID for model not found.')
            # Load model type based on json
            json_data = model_data['json']
            model_param_data = json.loads(json_data)
            model_type = model_param_data['type']
            if model_type == 'FairMetaModel':
                model = FairMetaModel.read_json(json_data)
            elif model_type == 'FairModel':
                model = FairModel.read_json(json_data)
            else:
                raise FairException('Unrecognized model type.')
        return model
    
    def store(self, model_or_metamodel):
        '''Take a model or metamodel and store in db'''
        m = model_or_metamodel
        # If incomplete and not ready for storage, throw error
        if not m.calculation_completed():
            raise FairException('Model has not been calculated and will not be stored.')
        # Export from model
        meta = json.loads(m.to_json())
        json_data = m.to_json()
        results = m.export_results()['Risk']
        # Write to database
        with sqlite3.connect(self._path) as conn:
            cursor = conn.cursor()
            # Write model data
            cursor.execute(
                '''INSERT OR REPLACE INTO model VALUES(?, ?, ?, ?)''',
                (meta['model_uuid'], meta['name'], meta['creation_date'], json_data)
            )
            # Write cached results
            cursor.execute(
                '''INSERT OR REPLACE INTO results VALUES(?, ?, ?, ?, ?)''',
                (meta['model_uuid'], results.mean(axis=0), results.std(axis=0), results.min(axis=0), results.max(axis=0))
            )
        # Vacuum database
        conn = sqlite3.connect(self._path)
        conn.execute("VACUUM")
        conn.commit()
        conn.close()
    
    def query(self, query, params=None):
        '''Allow queries on underlying database'''
        with sqlite3.connect(self._path) as conn:
            cursor = conn.cursor()
            if params:
                cursor.execute(query, params)
            else:
                cursor.execute(query)
            result = cursor.fetchall()
        return result
    
    
fdb = FairDatabase('./git_page/output.sqlite3')
fdb.store(meta1)
fdb.store(model1)
model = fdb.load('My Meta Model!')

model.export_results()

,Regular Model 1,Risk
0,3.153527e+09,3.153527e+09
1,2.049017e+09,2.049017e+09
2,4.281999e+09,4.281999e+09
3,1.263578e+09,1.263578e+09
4,5.322999e+09,5.322999e+09
5,7.445066e+08,7.445066e+08
6,5.250487e+09,5.250487e+09
7,1.283660e+09,1.283660e+09
8,1.439058e+09,1.439058e+09
9,3.189226e+09,3.189226e+09


In [7]:
model.calculate_all()
model.export_results()

,Regular Model 1,Risk
0,3.153527e+09,6.307055e+09
1,2.049017e+09,4.098034e+09
2,4.281999e+09,8.563998e+09
3,1.263578e+09,2.527156e+09
4,5.322999e+09,1.064600e+10
5,7.445066e+08,1.489013e+09
6,5.250487e+09,1.050097e+10
7,1.283660e+09,2.567319e+09
8,1.439058e+09,2.878116e+09
9,3.189226e+09,6.378451e+09
